# Greenest Pixel workflow and notebook

### Workflow

In [ ]:
from airflow.operators import CompressFileSensor
from cdcol_utils import other_utils
#coding=utf8
import airflow
from airflow.models import DAG
from airflow.operators import CDColQueryOperator, CDColFromFileOperator, CDColReduceOperator
from airflow.operators.python_operator import PythonOperator
from cdcol_utils import dag_utils, queue_utils, other_utils
from airflow.utils.trigger_rule import TriggerRule

from datetime import timedelta
from pprint import pprint

_params = {'minValid': 1, 'normalized': False, 'lat': (2, 3), 'lon': (-74, -73), 'products': [{'name': 'LS8_OLI_LASRC', 'bands': ['pixel_qa', 'red', 'nir']}], 'time_ranges': [('2020-01-01', '2020-06-30')], 'execID': 'exec_6855', 'elimina_resultados_anteriores': True, 'genera_mosaico': True, 'owner': 'API-REST'}

_steps = {
    'mascara': {
        'algorithm': "mascara-landsat",
        'version': '1.0',
        'queue': queue_utils.assign_queue(
            input_type='multi_temporal',
            time_range=_params['time_ranges'][0]
        ),
        'params': {},
    },
    'greenest_pixel':{
        'algorithm': "greenest_pixel-wf",
        'version': '1.0',
        'queue': queue_utils.assign_queue(
            input_type='multi_temporal_unidad',
            time_range=_params['time_ranges'][0],
            unidades=len(_params['products'])
        ),
        'params': {
            'normalized': _params['normalized'],
            'minValid': _params['minValid'],
        },
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
    'mosaico': {
        'algorithm': "joiner",
        'version': '1.0',
        'queue': queue_utils.assign_queue(
            input_type='multi_area',
            lat=_params['lat'],
            lon=_params['lon']
        ),
        'params': {},
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
}

args = {
    'owner': _params['owner'],
    'start_date': airflow.utils.dates.days_ago(2),
    'execID':_params['execID'],
    'product': _params['products'][0]['name']
}

dag = DAG(
    dag_id=args["execID"], default_args=args,
    schedule_interval=None,
    dagrun_timeout=timedelta(minutes=120)
)

mascara = dag_utils.queryMapByTile(
    lat=_params['lat'],
    lon=_params['lon'],
    time_ranges=_params['time_ranges'][0],
    algorithm=_steps['mascara']['algorithm'],
    version=_steps['mascara']['version'],
    product=_params['products'][0],
    params=_steps['mascara']['params'],
    queue=_steps['mascara']['queue'],
    dag=dag,
    task_id="mascara" + _params['products'][0]['name']
)

greenest_pixel = dag_utils.IdentityMap(
    mascara,
    algorithm=_steps['greenest_pixel']['algorithm'],
    product=_params['products'][0],
    version=_steps['greenest_pixel']['version'],
    task_id="greenest_pixel",
    queue=_steps['greenest_pixel']['queue'],
    dag=dag,
    delete_partial_results=_steps['greenest_pixel']['del_prev_result'], 
    to_tiff=not (_params['genera_mosaico'] and  queue_utils.get_tiles(_params['lat'], _params['lon']) > 1)
)

if _params['genera_mosaico'] and  queue_utils.get_tiles(_params['lat'], _params['lon']) > 1:
    mosaico = dag_utils.OneReduce(
        greenest_pixel,
        task_id="mosaico",
        algorithm=_steps['mosaico']['algorithm'],
        version=_steps['mosaico']['version'],
        queue=_steps['mosaico']['queue'],
        delete_partial_results=_steps['mosaico']['del_prev_result'],
        trigger_rule=TriggerRule.NONE_FAILED,
        dag=dag,
        to_tiff=True
    )
    
    mosaico
else:
    greenest_pixel

sensor_fin_ejecucion = CompressFileSensor(task_id='sensor_fin_ejecucion',poke_interval=60, soft_fail=True,mode='reschedule', queue='util', dag=dag) 
comprimir_resultados = PythonOperator(task_id='comprimir_resultados',provide_context=True,python_callable=other_utils.compress_results,queue='util',op_kwargs={'execID': args['execID']},dag=dag) 
sensor_fin_ejecucion >> comprimir_resultados 

# Mini-algorithms

In [ ]:
import numpy as np
print(product)
print ("Masking " + product['name'])
nodata=-9999
validValues=set()
if product['name']=="LS7_ETM_LEDAPS" or product['name'] == "LS5_TM_LEDAPS":
    validValues=[66,68,130,132]
elif product['name'] == "LS8_OLI_LASRC":
    validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
else:
    raise Exception("Este algoritmo sólo puede enmascarar LS7_ETM_LEDAPS, LS5_TM_LEDAPS o LS8_OLI_LASRC")

cloud_mask = np.isin(xarr0["pixel_qa"].values, validValues)
for band in product['bands']:
    print("entra a enmascarar")
    xarr0[band].values = np.where(np.logical_and(xarr0.data_vars[band] != nodata, cloud_mask), xarr0.data_vars[band], -9999)
output = xarr0

In [ ]:
import xarray as xr
import numpy as np
nodata=-9999
datos_red = xarr0["red"].values
datos_nir = xarr0["nir"].values
ndviTotal = []
for i in range(0,len(datos_red)):
    imagen1Roja = datos_red[i]
    imagen1nir = datos_nir[i]
    allNan=~np.isnan(imagen1Roja)
    mask_nan = np.logical_or(np.isnan(imagen1Roja),np.isnan(imagen1nir))
    #Calculo de ndvi a partir de bandas red y nir
    period_nvdi = np.true_divide(np.subtract(imagen1nir,imagen1Roja),np.add(imagen1nir,imagen1Roja))
    period_nvdi[mask_nan] = np.nan
    #Almacenamiento de resultados de calculo ndvi
    ndviTotal.append(period_nvdi)
    #print ndviTotal
    #print "tiempo" + str(i)
del datos_red,datos_nir

max_ndvi = np.nanmax(ndviTotal,0)

max_ndvi[max_ndvi>1]=np.nan
max_ndvi[max_ndvi<-1]=np.nan

print(max_ndvi.dtype)
ncoords=[]
xdims =[]
xcords={}
for x in xarr0.coords:
    if(x!='time'):
        ncoords.append( ( x, xarr0.coords[x]) )
        xdims.append(x)
        xcords[x]=xarr0.coords[x]
variables ={"MAX_NDVI": xr.DataArray(max_ndvi, dims=xdims,coords=ncoords)}
output=xr.Dataset(variables, attrs={'crs':xarr0.crs})
for x in output.coords:
    output.coords[x].attrs["units"]=xarr0.coords[x].units

In [ ]:
import xarray as xr
import glob, os,sys

output=None
xarrs=xarrs.values()
for _xarr in xarrs:
    if (output is None):
        output = _xarr
    else:
        output=output.combine_first(_xarr)

#output=xr.auto_combine(list(xarrs))
#output=xr.open_mfdataset("/source_storage/results/compuesto_de_medianas/compuesto-temporal-medianas-wf_1.0/*.nc")
#output=xr.merge(list(xarrs))